In [2]:
import os
os.chdir('/home/ubuntu/nndlproject/')

import open3d as o3d
import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.dataset import load_voxel_grid # voxelization function

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 1) Preprocessing the ModelNet datasets

Both the <a href='https://lmb.informatik.uni-freiburg.de/resources/datasets/ORION/modelnet40_manually_aligned.tar'>ModelNet40 (manually aligned)</a> and the <a href='http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip'>ModelNet10</a> databases come organized in folders like:

```
├── _label
    ├── _train
    |   ├── label_001.off
    |       ...
    └── _train
        └── label_101.off
            ...
    ...
```

Since our voxelization algorithm relies on `PyVista`, we cannot load directly `.off` files as meshes. We will need to convert them in the correct format with the help of another library, `Open3D`. 

In [3]:
# define modelnet directory
dataset_dir = '/dataNfs/modelnet40/'

# first, let's move all current models in a subdirectory off/

os.makedirs(os.path.join(dataset_dir,'off'),exist_ok=True)
# !mv /home/ubuntu/nndl-project/data/modelnet10/* /home/ubuntu/nndl-project/data/modelnet10/off/

In [4]:
# then we walk 
ROOT = os.path.join(dataset_dir,'off')

# make ply folder to store ply files
os.makedirs(os.path.join(dataset_dir,'ply'),exist_ok=True)

# converting loop
pbar = tqdm.tqdm(list(os.walk(ROOT)))

for path,subdirs,files in pbar:
    for f in files:
        if f[-4:] != '.off': continue # skip non .off files
        
        filename,extension = f.split('.')
        destination_path = path.replace("/off/", "/ply/")+'/{}.ply'.format(filename)
        if not os.path.exists(path.replace("/off/", "/ply/")): os.makedirs(path.replace("/off/", "/ply/"))
        
        mesh = o3d.io.read_triangle_mesh(path+'/'+f)
        o3d.io.write_triangle_mesh(destination_path, mesh)

100%|█████████████████████████████████████████| 123/123 [17:13<00:00,  8.40s/it]


Now, let's check the content of our newly creater `ply/` directory:

In [5]:
os.listdir(os.path.join(dataset_dir,'ply'))

['sink',
 'dresser',
 'lamp',
 'stool',
 'bookshelf',
 'person',
 'airplane',
 'tv_stand',
 'bed',
 'flower_pot',
 'range_hood',
 'bench',
 'desk',
 'bathtub',
 'bottle',
 'radio',
 'curtain',
 'table',
 'guitar',
 'xbox',
 'laptop',
 'door',
 'stairs',
 'sofa',
 'piano',
 'cup',
 'plant',
 'bowl',
 'monitor',
 'night_stand',
 'glass_box',
 'wardrobe',
 'cone',
 'car',
 'toilet',
 'keyboard',
 'tent',
 'chair',
 'vase',
 'mantel']

In [6]:
# list a few entries
os.listdir(os.path.join(dataset_dir,'ply','bathtub','test'))[:10]

['bathtub_0129.ply',
 'bathtub_0148.ply',
 'bathtub_0145.ply',
 'bathtub_0119.ply',
 'bathtub_0107.ply',
 'bathtub_0112.ply',
 'bathtub_0131.ply',
 'bathtub_0108.ply',
 'bathtub_0115.ply',
 'bathtub_0141.ply']

# 2) Preprocessing the voxel grid based on orientation class

## The `orientation_classes` table

When we want to train our model on multiple orientation, we can do so by defining a discrete set of orientation each object can occur in; we call these _orientation classes_, and our dataset of possible voxel grids input will be $\sum_k N_k O_k$ where $k$ runs on the number of object labels; we call these _classes_.

To speed up training in this case, we want to pre-compute the voxel grids and store them in `.npy` numpy files. So first, let's create the subdirectory:

In [7]:
os.makedirs(os.path.join(dataset_dir,'npy'),exist_ok=True)

Now, to define the orientation classes, we will use a `pandas.DataFrame`; we can custom define it later to tune our model and load it in our script, but for now we will go with a simple 4 classes for each object, each corresponding to a rotation of $90°$ around the $z$ axis.

We want the table to define each orientation class, link it with its associated class, a global index, and the $x,y,z$ rotation components in degrees.

In [8]:
# grab the labels from the other folder
labels = os.listdir(os.path.join(dataset_dir,'ply'))

# 4 orientation classes per label
class_id = [list(range(4)) for label in labels]

# create dataframe
orientation_classes = pd.DataFrame({
    'label' : labels,
    'class_id' : class_id
}).explode('class_id',ignore_index=True)

# now we have the index, the label, and the index of the orientation class relative to its parent class
orientation_classes.head(10)

,label,class_id
0,sink,0
1,sink,1
2,sink,2
3,sink,3
4,dresser,0
5,dresser,1
6,dresser,2
7,dresser,3
8,lamp,0
9,lamp,1


In [9]:
# we use that to compute the rotation columns
orientation_classes['rot_x']=0
orientation_classes['rot_y']=0
orientation_classes['rot_z']=90*orientation_classes['class_id']

orientation_classes.head(10)

,label,class_id,rot_x,rot_y,rot_z
0,sink,0,0,0,0
1,sink,1,0,0,90
2,sink,2,0,0,180
3,sink,3,0,0,270
4,dresser,0,0,0,0
5,dresser,1,0,0,90
6,dresser,2,0,0,180
7,dresser,3,0,0,270
8,lamp,0,0,0,0
9,lamp,1,0,0,90


In [10]:
# save this table, use it with script later
orientation_classes.to_csv('orientation_classes.csv')

## The voxelization

Now, we walk through all the `.ply` files, and use the orientation classes table as a reference to generate voxel grids. The following code takes quite a while to complete the voxelization, so we will run the script overnight, but here is its working:

In [11]:
# long, do not execute! will run the script separately overnight
if False:

    N=30

    ply_root = os.path.join(dataset_dir,'ply')


    pbar = tqdm.tqdm(list(os.walk(ply_root)))

    for root,subdirs,files in pbar:
        for f in files:
            if f[-4:] != '.ply' : continue
            
            label = root.split('/')[-2]
            destination_root = root.replace("/ply/", "/npy/")

            for index,row in orientation_classes[orientation_classes['label']==label].iterrows():

                pbar.set_description("Processing {}, orientation class {}".format(f,row['class_id']))

                # make dir for orientation class
                destination_path = os.path.join(destination_root,str(row['class_id']))
                if not os.path.exists(destination_path): os.makedirs(destination_path)

                # grab rotations
                rot_xyz = row[['rot_x','rot_y','rot_z']].values

                # voxelize
                array=load_voxel_grid(
                    os.path.join(root,f), # path of the original ply mesh
                    N,                    # resolution
                    *rot_xyz,             # rotation applied before voxelization
                    add_channel_dim=True)
                
                # save array
                file_destination_path = os.path.join(destination_path,f.split('.')[0]+'.npy')
                np.save(file_destination_path,array)
            


# 3) Generate metadata table

When we have the `.npy` files as well, we are now able to generate a metadata file that will be used by the `DataLoader` class to access these files.
The table will contain the following fields:

| path | split | label | label_id | orientation_class | orientation_class_id | rot_x | rot_y | rot_z |
|------|-------|-------|----------|-------------------|----------------------|-------|-------|-------|

We will list both the `.ply` files, that will have no orientation class predefined, and the `.npy` files, with their corresponding orientation class and rotation.

In [12]:
# first create a quick lookup table for the labels, from the orientation_classes table

lookup_table = pd.DataFrame({
    'label': labels,
    'label_id' : range(len(labels))}).set_index('label')

lookup_table

,label_id
label,
sink,0
dresser,1
lamp,2
stool,3
bookshelf,4
person,5
airplane,6
tv_stand,7
bed,8


In [13]:
# then we create the list of fields to populate
path = []
split = []
label = []
label_id = []
orientation_class = []
orientation_class_id = []
rot_x = []
rot_y = []
rot_z = []


for root,subdir,files in os.walk(dataset_dir):
    for f in files:

        # add entries for .ply files
        if f[-4:] == '.ply':
            
            path.append(os.path.join(root,f))
            split.append(root.split('/')[-1])
            label.append(root.split('/')[-2])
            label_id.append(lookup_table.loc[root.split('/')[-2]].item())
            # no orientation info
            orientation_class.append(None)
            orientation_class_id.append(None)
            rot_x.append(None)
            rot_y.append(None)
            rot_z.append(None)

        # add entries for .npy files
        elif f[-4:] == '.npy':
            
            path.append(os.path.join(root,f))
            # in this case we add the orientation class subdir
            split.append(root.split('/')[-2])
            label.append(root.split('/')[-3]) 
            label_id.append(lookup_table.loc[label[-1]].item())

            # orientation class is the id relative to the class: 0,1,2,3...n_orientation_classes_for_label
            orientation_class.append(int(root.split('/')[-1]))
            # orientation class id is the global index wrt the total number of orientation classes
            orientation_entry = orientation_classes[(orientation_classes['label']==label[-1]) * (orientation_classes['class_id']==orientation_class[-1])]
            orientation_class_id.append(orientation_entry.index.item())
            rot_x.append(orientation_entry['rot_x'].item())
            rot_y.append(orientation_entry['rot_y'].item())
            rot_z.append(orientation_entry['rot_z'].item())

        # skip everything else
        else:
            continue

# create the dataframe

metadata = pd.DataFrame(
    {
        'path' : path,
        'split' : split,
        'label' : label,
        'label_id' : label_id,
        'orientation_class' : orientation_class,
        'orientation_class_id' : orientation_class_id,
        'rot_x' : rot_x,
        'rot_y' : rot_y, 
        'rot_z' : rot_z
    }
).astype( # cast as types
    {
        'path' : 'str',
        'split' : 'str',
        'label' : 'str',
        'label_id' : 'Int32',
        'orientation_class' : 'Int32',
        'orientation_class_id' : 'Int32',
        'rot_x' : 'float',
        'rot_y' : 'float', 
        'rot_z' : 'float'
    }
)

In [14]:
# save it to disk
metadata.to_parquet(os.path.join(dataset_dir,'metadata.parquet')) # parquet is faster and preserves the index and dtypes

# take a look at the resulting table
metadata

,path,split,label,label_id,orientation_class,orientation_class_id,rot_x,rot_y,rot_z
0,/dataNfs/modelnet40/ply/sink/test/sink_0139.ply,test,sink,0,<NA>,<NA>,NaN,NaN,NaN
1,/dataNfs/modelnet40/ply/sink/test/sink_0136.ply,test,sink,0,<NA>,<NA>,NaN,NaN,NaN
2,/dataNfs/modelnet40/ply/sink/test/sink_0134.ply,test,sink,0,<NA>,<NA>,NaN,NaN,NaN
3,/dataNfs/modelnet40/ply/sink/test/sink_0137.ply,test,sink,0,<NA>,<NA>,NaN,NaN,NaN
4,/dataNfs/modelnet40/ply/sink/test/sink_0142.ply,test,sink,0,<NA>,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12306,/dataNfs/modelnet40/ply/mantel/train/mantel_00...,train,mantel,39,<NA>,<NA>,NaN,NaN,NaN
12307,/dataNfs/modelnet40/ply/mantel/train/mantel_02...,train,mantel,39,<NA>,<NA>,NaN,NaN,NaN
12308,/dataNfs/modelnet40/ply/mantel/train/mantel_00...,train,mantel,39,<NA>,<NA>,NaN,NaN,NaN
12309,/dataNfs/modelnet40/ply/mantel/train/mantel_00...,train,mantel,39,<NA>,<NA>,NaN,NaN,NaN


Now we can perform all sorts of filtering and subsetting, which will be useful when defining the `DataSet` class. For example, when instructed to work with `.npy` files, its internal reference will be the subset:

In [15]:
# load it
metadata = pd.read_parquet(os.path.join(dataset_dir,'metadata.parquet')) 

# subset on the npy files
metadata[metadata['path'].str.contains('/npy/')]

,path,split,label,label_id,orientation_class,orientation_class_id,rot_x,rot_y,rot_z


In [16]:
# subset on npy files and split 
metadata[metadata['path'].str.contains('/npy/') * (metadata['split']=='test')]

,path,split,label,label_id,orientation_class,orientation_class_id,rot_x,rot_y,rot_z


In [17]:
# load a sample and plot it

sample = metadata[metadata['path'].str.contains('/npy/')].sample(1)

%matplotlib Widget

vox_grid = np.load(sample['path'].item())

ax = plt.figure().add_subplot(projection='3d')
ax.voxels(vox_grid[0], edgecolor='k') # subset on channel dimension
ax.set_title('Label: {}, z rotation: {}'.format(sample['label'].item(),sample['rot_z'].item()))

plt.show()

ValueError: a must be greater than 0 unless no samples are taken